In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
#from vis_utils import *
import random;
import math;
import matplotlib.pyplot as plt

import os
print(os.listdir("../input"))

In [ ]:
sample = pd.read_csv("../input/sample_submission.csv") 
sample.head()


In [ ]:
test = pd.read_csv("../input/test.csv") 
test.head()

In [ ]:
test.shape[0]

In [ ]:
train = pd.read_csv("../input/train.csv") 
train.head()


In [ ]:
print(torch.__version__)
!python --version

In [ ]:
class Dataset(object):
    """An abstract class representing a Dataset.
    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [ ]:
class digitDataset(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.data.iloc[index, 1:].values.astype(np.uint8).reshape((1, 28, 28))
        label = self.data.iloc[index, 0]
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
class digitDatasetTrain(Dataset):
    
    def __init__(self, file_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.data.iloc[index, 0:].values.astype(np.uint8).reshape((1, 28, 28))
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image

In [ ]:
train_dataset = digitDataset('../input/train.csv', transform=None)
test_dataset = digitDatasetTrain('../input/test.csv', transform=None)

In [ ]:
train_dataset.__getitem__(0)
test_dataset.__getitem__(0)

In [ ]:
class NeuralNet(nn.Module):
    """A Neural Network with a hidden layer"""
    def __init__(self, input_size,hidden_size,output_size):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        output = self.layer1(x)
        output = self.relu(output)
        output = self.layer2(output)
        return output

In [ ]:
num_epochs = 5;
batch_size = 100;
learning_rate = 0.01;

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True);



In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=28000,
                                         shuffle=False);

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
#instance of the Conv Net
cnn = CNN();
#loss function and optimizer
criterion = nn.CrossEntropyLoss();
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate);

In [ ]:
losses = [];
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.float())
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item());
        
        if (i+1) % 100 == 0:
            print ('Epoch : %d/%d, Iter : %d/%d,  Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

In [ ]:
cnn

In [ ]:

for i, (images) in enumerate(test_loader):
        images = Variable(images.float())
        outputs = cnn(images)


In [ ]:
print(outputs.size())
print(outputs.type())
print(outputs.dim())

In [ ]:
print(outputs[[27999]])
print(outputs[:2, :])
outarray = outputs.detach().numpy()
outarray[[0]]

In [ ]:
outarray2 = outarray.argmax(1)

In [ ]:
outarray2.shape

In [ ]:
outarray2[0:10]

In [ ]:
out3 = outarray2

In [ ]:
out3[0:10]

In [ ]:
y = np.arange(28000)
y1 = y + 1
print(y1[0:10])
print(out3[0:10])
out3.shape

In [ ]:
preds = np.column_stack((y1,out3))
preds = preds.astype(int)
print(preds.size)
preds[:, :]

In [ ]:
np.savetxt("submission1.csv", preds, header="ImageId,Label", delimiter=",",comments='', fmt='%i')